In [1]:
# -*- coding: utf-8 -*-
"""
NiftyDataMining.py

This is script to load the NSE data, apply technical indicators
Moreover, it fetches data of other indices using multiple predefined CSVs and create a single data frame
"""
import pandas as pd
import gc
from talib import RSI, BBANDS, MACD, EMA, STOCH, ADX
import matplotlib.pyplot as plt
from nsepy import get_history
from datetime import date


In [2]:
#this is reference start date before EMA 200 days will fetch null
ref_startdate = date(2004,1,1)

#actual start date and end date from where the Nifty model to be built
startdate = date(2005,1,1)
enddate = date(2019,6,21)

df=pd.DataFrame(get_history(symbol="NIFTY",start = ref_startdate, end = enddate,index = True))
print(df.head(2))

               Open     High      Low    Close     Volume      Turnover
Date                                                                   
2004-01-01  1880.35  1917.05  1880.35  1912.25  102551958  2.846700e+10
2004-01-02  1912.25  1951.70  1911.05  1946.05  117079420  3.407720e+10


In [3]:
#calculating technical indicators such as EMA-200, EMA-100, EMA-50, EMA-21, EMA-5, MACD, RSI-14, Bollinger Bands
EMA_200 = EMA(df['Close'],200)
EMA_100 = EMA(df['Close'],100)
EMA_50 = EMA(df['Close'],50)
EMA_21 = EMA(df['Close'],21)
EMA_5 = EMA(df['Close'],5)
df['EMA-200']=EMA_200
df['EMA-100']=EMA_100
df['EMA-50']=EMA_50
df['EMA-21']=EMA_21
df['EMA-5']=EMA_5
df['MACD'], df['macdEMA'], df['macdHistory'] = MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['RSI-14']=RSI(df['Close'],timeperiod=14)
df['BollingerUpperBand'], df['BollingerMiddleBand'], df['BollingerLowerBand'] = BBANDS(df['Close'], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)

print(df.tail(2))

                Open      High       Low     Close     Volume      Turnover  \
Date                                                                          
2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954  1.883911e+11   
2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388  2.334613e+11   

                 EMA-200       EMA-100        EMA-50        EMA-21  \
Date                                                                 
2019-06-20  11202.010736  11459.610876  11665.894631  11791.634812   
2019-06-21  11207.205654  11464.848284  11668.177195  11785.495284   

                   EMA-5       MACD    macdEMA  macdHistory     RSI-14  \
Date                                                                     
2019-06-20  11775.159539  34.006803  66.882723   -32.875920  53.020715   
2019-06-21  11758.139693  24.843553  58.474889   -33.631337  48.011949   

            BollingerUpperBand  BollingerMiddleBand  BollingerLowerBand  
Date                           

In [4]:
df.index = (pd.to_datetime(df.index))

df= df[startdate:]
#Importing Brent Crude price Data from CSV"""
df1=pd.read_csv("BrentCrude.csv")
print(df1.head(2))

        Date  Brent Crude Price
0  21-Jun-19              65.31
1  20-Jun-19              64.45


In [5]:

# Convert column name Date from String datatype to Date datatype"""
df1['Date']=pd.to_datetime(df1['Date'])

#Sort the Sheet based on Date ascending order. This is one of pre-requisite to merge different dataframe.
#Dataframe that has to be merged should be sorted by join columns. In this case, join column is date.
#Hence, it is sorted based on Date column
df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date      Open      High       Low     Close     Volume  \
3587 2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954   
3588 2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388   

          Turnover       EMA-200       EMA-100        EMA-50        EMA-21  \
3587  1.883911e+11  11202.010736  11459.610876  11665.894631  11791.634812   
3588  2.334613e+11  11207.205654  11464.848284  11668.177195  11785.495284   

             EMA-5       MACD    macdEMA  macdHistory     RSI-14  \
3587  11775.159539  34.006803  66.882723   -32.875920  53.020715   
3588  11758.139693  24.843553  58.474889   -33.631337  48.011949   

      BollingerUpperBand  BollingerMiddleBand  BollingerLowerBand  \
3587        11882.455989             11742.03        11601.604011   
3588        11836.725120             11722.19        11607.654880   

      Brent Crude Price  
3587              64.45  
3588              65.31  


In [6]:
# Importing Dow Historical price Data from CSV. And perform similar action like BrentCrude price data"""
df1=pd.read_csv("DowHistorical.csv")
df1['Date']=pd.to_datetime(df1['Date'])
df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date      Open      High       Low     Close     Volume  \
3587 2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954   
3588 2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388   

          Turnover       EMA-200       EMA-100        EMA-50    ...      \
3587  1.883911e+11  11202.010736  11459.610876  11665.894631    ...       
3588  2.334613e+11  11207.205654  11464.848284  11668.177195    ...       

             EMA-5       MACD    macdEMA  macdHistory     RSI-14  \
3587  11775.159539  34.006803  66.882723   -32.875920  53.020715   
3588  11758.139693  24.843553  58.474889   -33.631337  48.011949   

      BollingerUpperBand  BollingerMiddleBand  BollingerLowerBand  \
3587        11882.455989             11742.03        11601.604011   
3588        11836.725120             11722.19        11607.654880   

      Brent Crude Price  Dow Price  
3587              64.45  26,753.17  
3588              65.31  26,719.13  

[2 rows x 21 columns]


In [7]:
print(df.tail(2))

           Date      Open      High       Low     Close     Volume  \
3587 2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954   
3588 2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388   

          Turnover       EMA-200       EMA-100        EMA-50    ...      \
3587  1.883911e+11  11202.010736  11459.610876  11665.894631    ...       
3588  2.334613e+11  11207.205654  11464.848284  11668.177195    ...       

             EMA-5       MACD    macdEMA  macdHistory     RSI-14  \
3587  11775.159539  34.006803  66.882723   -32.875920  53.020715   
3588  11758.139693  24.843553  58.474889   -33.631337  48.011949   

      BollingerUpperBand  BollingerMiddleBand  BollingerLowerBand  \
3587        11882.455989             11742.03        11601.604011   
3588        11836.725120             11722.19        11607.654880   

      Brent Crude Price  Dow Price  
3587              64.45  26,753.17  
3588              65.31  26,719.13  

[2 rows x 21 columns]


In [8]:
""" Importing FTSE Historical price Data from CSV. And perform similar action like BrentCrude price data"""
df1=pd.read_csv("FTSE_Historical.csv")
df1['Date']=pd.to_datetime(df1['Date'])
df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date      Open      High       Low     Close     Volume  \
3587 2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954   
3588 2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388   

          Turnover       EMA-200       EMA-100        EMA-50    ...     \
3587  1.883911e+11  11202.010736  11459.610876  11665.894631    ...      
3588  2.334613e+11  11207.205654  11464.848284  11668.177195    ...      

           MACD    macdEMA  macdHistory     RSI-14  BollingerUpperBand  \
3587  34.006803  66.882723   -32.875920  53.020715        11882.455989   
3588  24.843553  58.474889   -33.631337  48.011949        11836.725120   

      BollingerMiddleBand  BollingerLowerBand  Brent Crude Price  Dow Price  \
3587             11742.03        11601.604011              64.45  26,753.17   
3588             11722.19        11607.654880              65.31  26,719.13   

          FTSE  
3587  7,424.44  
3588  7,407.50  

[2 rows x 22 columns]


In [9]:
#Importing Hang Seng Historical price Data from CSV. And perform similar action like BrentCrude price data"""
df1=pd.read_csv("HangSeng_Historical.csv")
df1['Date']=pd.to_datetime(df1['Date'])
df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date      Open      High       Low     Close     Volume  \
3587 2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954   
3588 2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388   

          Turnover       EMA-200       EMA-100        EMA-50       ...         \
3587  1.883911e+11  11202.010736  11459.610876  11665.894631       ...          
3588  2.334613e+11  11207.205654  11464.848284  11668.177195       ...          

        macdEMA  macdHistory     RSI-14  BollingerUpperBand  \
3587  66.882723   -32.875920  53.020715        11882.455989   
3588  58.474889   -33.631337  48.011949        11836.725120   

      BollingerMiddleBand  BollingerLowerBand  Brent Crude Price  Dow Price  \
3587             11742.03        11601.604011              64.45  26,753.17   
3588             11722.19        11607.654880              65.31  26,719.13   

          FTSE  Hang Seng Price  
3587  7,424.44        28,202.14  
3588  7,407.50        28,550.43  

[2 rows x 23 c

In [10]:
#Importing USD_INR Historical price Data from CSV. And perform similar action like BrentCrude price data"""
df1=pd.read_csv("USD_INR_Historical.csv")
df1['Date']=pd.to_datetime(df1['Date'])

df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date      Open      High       Low     Close     Volume  \
3587 2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954   
3588 2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388   

          Turnover       EMA-200       EMA-100        EMA-50      ...        \
3587  1.883911e+11  11202.010736  11459.610876  11665.894631      ...         
3588  2.334613e+11  11207.205654  11464.848284  11668.177195      ...         

      macdHistory     RSI-14  BollingerUpperBand  BollingerMiddleBand  \
3587   -32.875920  53.020715        11882.455989             11742.03   
3588   -33.631337  48.011949        11836.725120             11722.19   

      BollingerLowerBand  Brent Crude Price  Dow Price      FTSE  \
3587        11601.604011              64.45  26,753.17  7,424.44   
3588        11607.654880              65.31  26,719.13  7,407.50   

      Hang Seng Price  USD-INR price  
3587        28,202.14         69.520  
3588        28,550.43         69.615  

[2 rows 

In [11]:
#Importing India VIX(Volatility Index) Historical Data from CSV. And perform similar action like BrentCrude price data"""
df1=pd.read_csv("India VIX Historical Data.csv")
df1['Date']=pd.to_datetime(df1['Date'])

df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date      Open      High       Low     Close     Volume  \
3587 2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954   
3588 2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388   

          Turnover       EMA-200       EMA-100        EMA-50  \
3587  1.883911e+11  11202.010736  11459.610876  11665.894631   
3588  2.334613e+11  11207.205654  11464.848284  11668.177195   

              ...              RSI-14  BollingerUpperBand  \
3587          ...           53.020715        11882.455989   
3588          ...           48.011949        11836.725120   

      BollingerMiddleBand  BollingerLowerBand  Brent Crude Price  Dow Price  \
3587             11742.03        11601.604011              64.45  26,753.17   
3588             11722.19        11607.654880              65.31  26,719.13   

          FTSE  Hang Seng Price  USD-INR price  Volatility Index VIX  
3587  7,424.44        28,202.14         69.520                 14.00  
3588  7,407.50        28,550.43 

In [12]:

df2=pd.read_excel("FII-Index.xlsx")
df2['Date']=pd.to_datetime(df2['Date'])
#df2.dropna(inplace=True)
df2=df2.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df2,on='Date')

In [13]:
print(df.tail(2))

           Date      Open      High       Low     Close     Volume  \
3587 2019-06-20  11653.65  11843.50  11635.05  11831.75  442016954   
3588 2019-06-21  11827.60  11827.95  11705.10  11724.10  468430388   

          Turnover       EMA-200       EMA-100        EMA-50  \
3587  1.883911e+11  11202.010736  11459.610876  11665.894631   
3588  2.334613e+11  11207.205654  11464.848284  11668.177195   

            ...          BollingerMiddleBand  BollingerLowerBand  \
3587        ...                     11742.03        11601.604011   
3588        ...                     11722.19        11607.654880   

      Brent Crude Price  Dow Price      FTSE  Hang Seng Price  USD-INR price  \
3587              64.45  26,753.17  7,424.44        28,202.14         69.520   
3588              65.31  26,719.13  7,407.50        28,550.43         69.615   

      Volatility Index VIX  FII Index Future Net  FII Index Options  
3587                 14.00               -236.99            1075.72  
3588      

In [14]:
df = df.set_index('Date') 
df.rename(columns={'Close': 'Nifty'}, inplace=True)
print(df.head(2))

               Open     High      Low    Nifty    Volume      Turnover  \
Date                                                                     
2005-01-03  2080.00  2118.60  2080.00  2115.00  70506865  2.375100e+10   
2005-01-04  2116.95  2120.15  2100.55  2103.75  72718302  2.416130e+10   

                EMA-200      EMA-100       EMA-50     EMA-21  \
Date                                                           
2005-01-03  1807.768326  1863.544641  1951.264565  2031.2347   
2005-01-04  1810.713417  1868.301183  1957.244386  2037.8270   

                  ...         BollingerMiddleBand  BollingerLowerBand  \
Date              ...                                                   
2005-01-03        ...                     2079.25         2041.161051   
2005-01-04        ...                     2085.73         2044.340649   

            Brent Crude Price    Dow Price      FTSE  Hang Seng Price  \
Date                                                                    
2005-01

In [15]:
df = df.drop(["Open", "High", "Low", "macdEMA", "macdHistory"], axis=1)
print(df.head(2))



              Nifty    Volume      Turnover      EMA-200      EMA-100  \
Date                                                                    
2005-01-03  2115.00  70506865  2.375100e+10  1807.768326  1863.544641   
2005-01-04  2103.75  72718302  2.416130e+10  1810.713417  1868.301183   

                 EMA-50     EMA-21        EMA-5      MACD     RSI-14  \
Date                                                                   
2005-01-03  1951.264565  2031.2347  2083.265125  43.08027  78.071485   
2005-01-04  1957.244386  2037.8270  2090.093417  43.44056  73.511082   

                  ...          BollingerMiddleBand  BollingerLowerBand  \
Date              ...                                                    
2005-01-03        ...                      2079.25         2041.161051   
2005-01-04        ...                      2085.73         2044.340649   

            Brent Crude Price    Dow Price      FTSE Hang Seng Price  \
Date                                             

In [16]:
#Save the mined data in csv format
df.to_csv('NSE-Data-new back upcsv.csv')


In [17]:

#delete all the variable and free up the space
del df, df1, df2, enddate, ref_startdate, startdate
gc.collect()

183695